In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd 
import pickle 

import data_utils
import sys

import pre_process

In [ ]:
pandas_readable_data = '/data2/data/zebrahim/AI/lendingClub-CS541/pandas_readable.csv'

In [ ]:
data_frame = pd.read_csv(pandas_readable_data)

In [ ]:
np.unique(np.asarray(data_frame.dtypes))

In [ ]:
X_str_idx = []
X_num_idx = []
for i in range(len(data_frame.columns[:-1])):
    if data_frame.dtypes[i] == 'O':
        X_str_idx.append(i)
    else:
        X_num_idx.append(i) 
        

In [ ]:
y = np.asarray(data_frame.iloc[:,-1]).astype('S')

In [ ]:
X_str = np.asarray(data_frame.iloc[:,X_str_idx]).astype('S')
X_num = np.asarray(data_frame.iloc[:,X_num_idx]).astype('float32')

X_str_idx = np.asarray(X_str_idx)
X_num_idx = np.asarray(X_num_idx)

In [ ]:
y.shape, X_str.shape, X_num.shape

In [ ]:
# Set the float format
float_formater = lambda x: "%.2f" % x
np.set_printoptions(formatter = {'float_kind' : float_formater})

In [ ]:
X_str[:, 0:20]

In [ ]:
data_frame

# data Preprocessing
## Step1

In [ ]:
#making a zeror matrix and will put one if a data for the specific cell is unknown
Xbinary_str = np.zeros(X_str.shape)
Xbinary_num = np.zeros(X_num.shape)

#including tuples (i,j) which are cells with unknown info
null_idx_Xbinary_str= []
null_idx_Xbinary_num= []


## Dealing with string columns
We have figured out that only three columns among string columns have a problem. They are 11,21,59

In [ ]:
#column 11
corspnd_idx = np.where(X_str_idx == 11)[0][0]
null_idx_Xbinary_str += [(i,corspnd_idx) for i in np.where(X_str[:,corspnd_idx] == 'n/a')[0]]

In [ ]:
#column 21
corspnd_idx = np.where(X_str_idx == 21)[0][0]
null_idx_Xbinary_str += [(i,corspnd_idx) for i in np.where(X_str[:,corspnd_idx] == 'nan')[0]]

In [ ]:
#column 59
corspnd_idx = np.where(X_str_idx == 59)[0][0]
null_idx_Xbinary_str += [(i,corspnd_idx) for i in np.where(X_str[:,corspnd_idx] == 'nan')[0]]

Dealing with integer, float columns

In [ ]:
#rows,cols = np.where(np.isnan(X_num))
#cnt = np.isnan(X_num).sum(axis=0)
#cols_big = np.where(cnt>1000)[0]
#rows_remain = np.unique([r for r, c in zip(rows, cols) if c not in cols_big])
#rows_remain.shape

In [ ]:
for i in X_num_idx:
    corrspnd_idx = np.where(X_num_idx==i)[0][0]
    tmp_str_col = np.asarray(X_num[:,corrspnd_idx]).astype('S')
    nan_idx = np.where(tmp_str_col == 'nan')[0]
    
    if nan_idx.shape[0]>0 :
        null_idx_Xbinary_num += [(j,corrspnd_idx) for j in nan_idx]

In [ ]:
col_idx_num = np.asarray(null_idx_Xbinary_num)[:,1]
row_idx_num = np.asarray(null_idx_Xbinary_num)[:,0]

col_idx_str = np.asarray(null_idx_Xbinary_str)[:,1]
row_idx_str = np.asarray(null_idx_Xbinary_str)[:,0]

In [ ]:
Xbinary_str[row_idx_str, col_idx_str] = 1
Xbinary_num[row_idx_num, col_idx_num] = 1

## deciding which columns and rows should be removed
Finding which columns should be removed

In [ ]:
sum_Xnum_col = Xbinary_num.sum(axis=0) #[Xbinary_num[:,i].sum() for i in range(len(X_num_idx))]
sum_Xstr_col = Xbinary_str.sum(axis=0) #[Xbinary_str[:,i].sum() for i in range(len(X_str_idx))]

In [ ]:
#if a column has more than 1K unknown cell, it will be removed
str_col_removal_idx = []    #idx in the whole dataset
str_col_removal_idx_binary = []
for i in range(len(sum_Xstr_col)):
    if sum_Xstr_col[i]>1000:
        str_col_removal_idx.append(X_str_idx[i])
        str_col_removal_idx_binary.append(i)
        
num_col_removal_idx = []
num_col_removal_idx_binary = []
for i in range(len(sum_Xnum_col)):
    if sum_Xnum_col[i]>1000:
        num_col_removal_idx.append(X_num_idx[i])
        num_col_removal_idx_binary.append(i)


In [ ]:
Xbinary_str = np.delete(Xbinary_str,str_col_removal_idx_binary,axis=1)
Xbinary_num = np.delete(Xbinary_num,num_col_removal_idx_binary,axis=1)

Finding which rows should be removed

In [ ]:
sum_Xstr_row = [Xbinary_str[i,:].sum() for i in range(Xbinary_str.shape[0])]
sum_Xnum_row = [Xbinary_num[i,:].sum() for i in range(Xbinary_num.shape[0])]

In [ ]:
row_removal_idx = []
for i in range(len(sum_Xstr_row)):
    if sum_Xstr_row[i]>0:
        row_removal_idx.append(i)
        
for i in range(len(sum_Xnum_row)):     
    if sum_Xnum_row[i]>0:
        row_removal_idx.append(i)
        

removing rows and columns

In [ ]:
#cols_keep = np.asarray(list(set(range(X_num.shape[1]))-set(num_col_removal_idx)))
#cols_keep.shape[0] - X_num.shape[1]
#num_col_removal_idx

In [ ]:
col_remove = [5,9,10,11,15,17,18,19,20,21,22,23,37,47,49,50,115]  # index of column needs to remove
for i in col_remove:
    loc = np.where(i == X_str_idx)[0][0]
    if loc not in str_col_removal_idx_binary:
        str_col_removal_idx_binary.append(loc)
        
    if i not in str_col_removal_idx:
        str_col_removal_idx.append(i)


In [ ]:
X_str = np.delete(X_str, str_col_removal_idx_binary, axis=1)
X_num = np.delete(X_num, num_col_removal_idx_binary, axis=1)

X_str = np.delete(X_str, row_removal_idx, axis=0)
X_num = np.delete(X_num, row_removal_idx, axis=0)

y = np.delete(y, np.asarray(row_removal_idx), axis=0)

In [ ]:
X_str_idx = [i for i in X_str_idx if i not in str_col_removal_idx]


## Step2

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

scaling all intege/float columns between (-1,1)

In [ ]:
scaler = MinMaxScaler()
for i in range(X_num.shape[1]):
    X_num[:,i] = scaler.fit_transform(X_num[:,i].reshape([-1,1])).flatten().astype('float64')

Encoding all columns included string

In [ ]:
#column 26 in origin data, or 4 in X_str = 'earliest_cr_line'
temp = np.asarray(data_frame.iloc[:,26]).astype('datetime64').astype('float64')
temp = np.delete(temp, row_removal_idx, axis = 0 )
X_str[:,4] =  scaler.fit_transform(temp.reshape([-1,1])).flatten().astype('float64')


In [ ]:
labelencoder_X = LabelEncoder()
for i in range(X_str.shape[1]):
    if i != 4:
        X_str[:,i] = labelencoder_X.fit_transform(X_str[:,i])

In [ ]:
print X_str[:,0], np.unique(X_str[:,0])
print X_str[:,1], np.unique(X_str[:,1])
print X_str[:,2], np.unique(X_str[:,2])
print X_str[:,3], np.unique(X_str[:,3])
#print X_str[:,4], np.unique(X_str[:,4])
print X_str[:,5], np.unique(X_str[:,5])
print X_str[:,6], np.unique(X_str[:,6])


In [ ]:
X_str.shape

In [ ]:
onehotencoder = OneHotEncoder(categorical_features=[0,1,2,3,5,6])
X_str = onehotencoder.fit_transform(X_str).toarray()

In [ ]:
X = np.concatenate((X_str, X_num), axis=1)

In [ ]:
data_split = np.asarray(data_frame.iloc[:,115]).astype('S')

In [ ]:
data_split = np.delete(data_split, row_removal_idx, axis =0 )

In [ ]:
data_split.shape

In [ ]:
y_ = np.zeros(y.shape)
pos_idx = np.where(y == 'good')[0]
y_[pos_idx] = 1

In [ ]:
data = {}
data['X'] = X
data['y'] = y_
data['split_info'] = data_split

In [ ]:
import pickle
with open ('data.p', 'w') as fout:
    pickle.dump(data, fout)